In [1]:
from grad_cam import *
import torchvision.models as models
import torch.nn as nn
import matplotlib.cm as cm
import torch
import cv2
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset
from efficientnet_pytorch import EfficientNet
import os.path as osp
from PIL import Image
import numbers
from torchvision.transforms.functional import pad as PAD
import pandas as pd
import os
import natsort
import torch.nn.functional as F
import numpy
from torchvision.transforms import ToPILImage,ToTensor
ToPILImage = ToPILImage()
ToTensor = ToTensor()
input_size=1024

In [2]:
class ImageSet(Dataset):
    def __init__(self, main_dir, transform):
        self.main_dir = main_dir
        self.transform = transform
        all_imgs = os.listdir(main_dir)
        self.total_imgs = natsort.natsorted(all_imgs)

    def __len__(self):
        return len(self.total_imgs)

    def __getitem__(self, idx):
        img_loc = os.path.join(self.main_dir, self.total_imgs[idx])
        img_name = self.total_imgs[idx]
        image = Image.open(img_loc).convert("RGB")
        tensor_image = self.transform(image)
        return tensor_image,img_name
    

In [3]:
def get_padding(image):    
    w, h = image.size
    max_wh = np.max([w, h])
    h_padding = (max_wh - w) / 2
    v_padding = (max_wh - h) / 2
    l_pad = h_padding if h_padding % 1 == 0 else h_padding+0.5
    t_pad = v_padding if v_padding % 1 == 0 else v_padding+0.5
    r_pad = h_padding if h_padding % 1 == 0 else h_padding-0.5
    b_pad = v_padding if v_padding % 1 == 0 else v_padding-0.5
    padding = (int(l_pad), int(t_pad), int(r_pad), int(b_pad))
    return padding

def get_cv2_padding(image):    
    (h,w,d) = raw_image.shape
    max_wh = np.max([w, h])
    h_padding = (max_wh - w) / 2
    v_padding = (max_wh - h) / 2
    l_pad = h_padding if h_padding % 1 == 0 else h_padding+0.5
    t_pad = v_padding if v_padding % 1 == 0 else v_padding+0.5
    r_pad = h_padding if h_padding % 1 == 0 else h_padding-0.5
    b_pad = v_padding if v_padding % 1 == 0 else v_padding-0.5
    padding = (int(l_pad), int(t_pad), int(r_pad), int(b_pad))
    return padding

class NewPad(object):
    def __init__(self, 
                 fill=(39,39,39), 
                 padding_mode='constant',
                ):
        assert isinstance(fill, (numbers.Number, str, tuple))
        assert padding_mode in ['constant', 'edge', 'reflect', 'symmetric']
        self.fill = fill
        self.padding_mode = padding_mode
        
    def __call__(self, img):
        return PAD(img, get_padding(img), self.fill, self.padding_mode)
    def __repr__(self):
        return self.__class__.__name__ + '(padding={0}, fill={1}, padding_mode={2})'.format(self.fill, 
                                                                                            self.padding_mode)
    
def invTrans(image):
    transformation = transforms.Compose([transforms.Normalize(mean = [ 0., 0., 0. ],
                                                     std = [ 1/0.1402, 1/0.1402, 1/0.1402 ]),
                                         transforms.Normalize(
                                             mean = [ -0.1523, -0.1523, -0.1523 ],
                                             std = [ 1., 1., 1. ]),
                               ])
    image = transformation(image)
    return image

def preprocess(inputs):
    image = inputs
    image = invTrans(inputs)
    image = ToPILImage(image)
    return image

ImgTransform = transforms.Compose([
        NewPad(),
        transforms.Resize(input_size),
        transforms.ToTensor(),
    ])

In [4]:
Image_Sets = {x:ImageSet(os.path.join('./grad-cam-pytorch/outputs',x),
                         transform=ImgTransform) for x in ['Direct', 'MURA3']}

In [5]:
Image_Sets['Originals'] = ImageSet('../../data/AnimalDatasetV2Original/',transform=ImgTransform)

In [6]:
# MetaData = pd.read_csv('./With Class Probability.csv',dtype={'Image No.':str})
# MetaData.head()

In [7]:
with open('./Meta For Animal Dataset Ver.2.xlsx',mode='rb') as f:
    MetaData = pd.read_excel(f,sheet_name="MVF FN", dtype={"Image No.":str})

In [8]:
MetaData.head()

,Image Finding,Image No.,MURA Wrongs,MURA_Localization
0,Avulsion,187,FN,Incorrect
1,Destructive Lesion,031,FN,Incorrect
2,Destructive Lesion,030,FN,Correct
3,Destructive Lesion,032,FN,Incorrect
4,Destructive Lesion,108,FN,Incorrect


In [9]:
MetaData['Image Finding'].unique()

array(['Avulsion', 'Destructive Lesion', 'DJD', 'Elbow Dysplasia',
       'Fracture', 'Hypertrophic Osteodystrophy', 'Luxation',
       'Mixed Lesion', 'Osteochondrosis', 'Productive Lesion',
       'Radial Hypoplasia', 'Retained Cartilage Core', 'Hardware',
       'Soft Tissue Mass', 'Soft Tissue Swelling'], dtype=object)

In [10]:
# MetaData.groupby(MetaData['Is_Normal']).count()

In [11]:
Images = {}
i = 0
for cat in ['Originals', 'Direct','MURA3']:
    Images[cat] = []
    dataloader = torch.utils.data.DataLoader(Image_Sets[cat])
    for i in range(0,len(dataloader.dataset.total_imgs)):
        [inputs,name] = dataloader.dataset[i]
        img = ToPILImage(inputs)
        Images[cat].append(img)

In [23]:
import matplotlib.pyplot as plt
font = {'fontname':'Times New Roman',
         'size'   : 22}

In [15]:
# for i in np.arange(0,500,20):
#     fig,axs = plt.subplots(20,3, figsize=(30,245))
#     plt.tight_layout()
#     for k in range(0,20):
#         axs[k,0].imshow(Images['Originals'][i])
#         axs[k,0].axis('off')
#         Ori_caption = MetaData.loc[i,'Image No.']+'\n'+MetaData.loc[i,'Image Finding']
#         axs[k,0].set_title(Ori_caption,**font)


#         axs[k,1].imshow(Images['Direct'][i])
#         axs[k,1].axis('off')
#         Direct_caption = "Direct Finetune\n"
#         Direct_caption = Direct_caption+ "Pr(Normal) = "+str(round(MetaData.loc[i,'Direct_Nor_Score']*100,ndigits=2))+'%'
#         Direct_caption = Direct_caption+ "    Pr(Abnormal) = "+str(round(MetaData.loc[i,'Direct_Abn_Score']*100,ndigits=2))+'%\n'
#         if (MetaData.loc[i,'Is_Normal'] == 'Normal') and (MetaData.loc[i,'Direct Correct']==1):
#             Direct_caption = Direct_caption+ 'Classification: Normal\nCorrect'
#         if (MetaData.loc[i,'Is_Normal'] == 'Normal') and (MetaData.loc[i,'Direct Correct']==0):
#             Direct_caption = Direct_caption+ 'Classification: Abnormal\nIncorrect'
#         if (MetaData.loc[i,'Is_Normal'] == 'Abnormal') and (MetaData.loc[i,'Direct Correct']==1):
#             Direct_caption = Direct_caption+ 'Classification: Abnormal\nCorrect'
#         if (MetaData.loc[i,'Is_Normal'] == 'Abnormal') and (MetaData.loc[i,'Direct Correct']==0):
#             Direct_caption = Direct_caption+ 'Classification: Normal\nIncorrect'
# #         print(Direct_caption)
#         axs[k,1].set_title(Direct_caption,**font)
        

#         axs[k,2].imshow(Images['MURA3'][i])
#         axs[k,2].axis('off')
#         MURA_caption = "MURA_Vet_Finetune\n"
#         MURA_caption = MURA_caption+ "Pr(Normal) = "+str(round(MetaData.loc[i,'MURA_Nor_Score']*100,ndigits=2))+'%'
#         MURA_caption = MURA_caption+ "    Pr(Abnormal) = "+str(round(MetaData.loc[i,'MURA_Abn_Score']*100,ndigits=2))+'%\n'
#         if (MetaData.loc[i,'Is_Normal'] == 'Normal') and (MetaData.loc[i,'MURA Correct']==1):
#             MURA_caption = MURA_caption+ 'Classification: Normal\nCorrect'
#         if (MetaData.loc[i,'Is_Normal'] == 'Normal') and (MetaData.loc[i,'MURA Correct']==0):
#             MURA_caption = MURA_caption+ 'Classification: Abnormal\nIncorrect'
#         if (MetaData.loc[i,'Is_Normal'] == 'Abnormal') and (MetaData.loc[i,'MURA Correct']==1):
#             MURA_caption = MURA_caption+ 'Classification: Abnormal\nCorrect'
#         if (MetaData.loc[i,'Is_Normal'] == 'Abnormal') and (MetaData.loc[i,'MURA Correct']==0):
#             MURA_caption = MURA_caption+ 'Classification: Normal\nIncorrect'
# #         print(MURA_caption)
#         axs[k,2].set_title(MURA_caption,**font)
#         i+=1
#     plt.savefig(os.path.join('./Grad-CAM_Results_Ver3_FN/',str(i-19)+'-'+str(i)))
#     plt.clf()
#     plt.close()

In [ ]:
for i in np.arange(0,500,20):
    fig,axs = plt.subplots(20,3, figsize=(30,245))
    plt.tight_layout()
    for k in range(0,20):
        axs[k,0].imshow(Images['Originals'][i])
        axs[k,0].axis('off')
        Ori_caption = MetaData.loc[i,'Image No.']+'\n'+MetaData.loc[i,'Image Finding']
        axs[k,0].set_title(Ori_caption,**font)


        axs[k,1].imshow(Images['Direct'][i])
        axs[k,1].axis('off')
        Direct_caption = "Direct Finetune\n"
        Direct_caption = Direct_caption+ "Pr(Normal) = "+str(round(MetaData.loc[i,'Direct_Nor_Score']*100,ndigits=2))+'%'
        Direct_caption = Direct_caption+ "    Pr(Abnormal) = "+str(round(MetaData.loc[i,'Direct_Abn_Score']*100,ndigits=2))+'%\n'
        if (MetaData.loc[i,'Is_Normal'] == 'Normal') and (MetaData.loc[i,'Direct Correct']==1):
            Direct_caption = Direct_caption+ 'Classification: Normal\nCorrect'
        if (MetaData.loc[i,'Is_Normal'] == 'Normal') and (MetaData.loc[i,'Direct Correct']==0):
            Direct_caption = Direct_caption+ 'Classification: Abnormal\nIncorrect'
        if (MetaData.loc[i,'Is_Normal'] == 'Abnormal') and (MetaData.loc[i,'Direct Correct']==1):
            Direct_caption = Direct_caption+ 'Classification: Abnormal\nCorrect'
        if (MetaData.loc[i,'Is_Normal'] == 'Abnormal') and (MetaData.loc[i,'Direct Correct']==0):
            Direct_caption = Direct_caption+ 'Classification: Normal\nIncorrect'
#         print(Direct_caption)
        axs[k,1].set_title(Direct_caption,**font)
        

        axs[k,2].imshow(Images['MURA3'][i])
        axs[k,2].axis('off')
        MURA_caption = "MURA_Vet_Finetune\n"
        MURA_caption = MURA_caption+ "Pr(Normal) = "+str(round(MetaData.loc[i,'MURA_Nor_Score']*100,ndigits=2))+'%'
        MURA_caption = MURA_caption+ "    Pr(Abnormal) = "+str(round(MetaData.loc[i,'MURA_Abn_Score']*100,ndigits=2))+'%\n'
        if (MetaData.loc[i,'Is_Normal'] == 'Normal') and (MetaData.loc[i,'MURA Correct']==1):
            MURA_caption = MURA_caption+ 'Classification: Normal\nCorrect'
        if (MetaData.loc[i,'Is_Normal'] == 'Normal') and (MetaData.loc[i,'MURA Correct']==0):
            MURA_caption = MURA_caption+ 'Classification: Abnormal\nIncorrect'
        if (MetaData.loc[i,'Is_Normal'] == 'Abnormal') and (MetaData.loc[i,'MURA Correct']==1):
            MURA_caption = MURA_caption+ 'Classification: Abnormal\nCorrect'
        if (MetaData.loc[i,'Is_Normal'] == 'Abnormal') and (MetaData.loc[i,'MURA Correct']==0):
            MURA_caption = MURA_caption+ 'Classification: Normal\nIncorrect'
#         print(MURA_caption)
        axs[k,2].set_title(MURA_caption,**font)
        i+=1
    plt.savefig(os.path.join('./Grad-CAM_Results_Ver3_FN/',str(i-19)+'-'+str(i)))
    plt.clf()
    plt.close()

In [ ]:
MetaData.loc[MetaData['Image No.']=='112']

In [19]:
with open('./Meta For Animal Dataset Ver.2.xlsx',mode='rb') as f:
    FP = pd.read_excel(f,sheet_name="MVF FP", dtype={"Image No.":str})

In [20]:
FP

,Image Finding,Image No.,MURA Wrongs
0,Normal,018,FP
1,Normal,019,FP
2,Normal,028,FP
3,Normal,029,FP
4,Normal,041,FP
...,...,...,...
56,Normal,446,FP
57,Normal,450,FP
58,Normal,469,FP
59,Normal,494,FP


In [36]:
fp_n = []
for i,r in FP.iterrows():
    n=int(r['Image No.'])
    fp_n.append(n)

fig,axs = plt.subplots(20,1,figsize=(10,245))
plt.tight_layout()


s=40
for k in range(0,20):
    axs[k].imshow(Images['MURA3'][fp_n[k+s]-1])
    axs[k].axis('off')
    caption = str(fp_n[k+s])
    axs[k].set_title(caption,**font)
plt.savefig(os.path.join('./Grad-CAM_Results_Ver3_FP','FP3'))
plt.clf()
plt.close()